In [128]:
### ALL CODES BELOW (AS A UNITY) WORK ONLY FOR THE "GOOGLE CLOUD INSTANCE" IN SQL CONNECTIONS (HOME OF MYSQLWORKBENCH)!
### This complete file NEEDS to be run before adding updated information to SQL database tables in the future.
### (For example: updated population, weather, or airports and flight information)
### Note: getting updated information would require running the separate files, as needed (i.e. weather to get current weather info)

#IMPORTANT:
### RUN ALL CELLS ONLY WHEN HAVE RUN SQL QUERY (= HAVE DELETED AND RECREATED SQL DATABASE AND TABLES) AND WANT
### BASIS INFORMATION ON WHICH COULD BE ADDED ON IN THE FUTURE.

In [ ]:
cities = ['City1', 'City2', 'City3'] #replace City1, City2...with actual city names

In [131]:
# install if needed:
# !pip install sqlalchemy
# !pip install pymysql

# libraries used:
import get_connection_string_GCP as gcs
import get_city_data as gcd
import get_population_data_GCP as gpd
import get_weather_data as gwd
import get_airport_codes as gac
import get_flight_arrivals as gfa
import pandas as pd
import requests

In [132]:
connection_string = gcs.get_connection_string()

## Get_city information

In [134]:
city_scraping_df=gcd.get_city_data(cities)

In [135]:
city_pop_scraping_df=gpd.get_population_data(cities)

### Countries

In [137]:
countries_df = city_scraping_df['country'].unique()
countries_df = pd.DataFrame({"country": countries_df})

In [138]:
countries_df.to_sql('countries',
                  if_exists='append',
                  con=connection_string,
                  index=False)

1

In [139]:
countries_from_sql = pd.read_sql("countries", con=connection_string)

### Cities

In [141]:
merged_countries_df = city_scraping_df.merge(countries_from_sql,
                                             on = "country",
                                             how = "left")

In [142]:
cities_df = merged_countries_df.drop(columns=["country"])

In [143]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

### Population

In [145]:
cities_from_sql = pd.read_sql("cities", con=connection_string)

In [146]:
merged_population = cities_from_sql.merge(city_pop_scraping_df,
                                   on = "city",
                                   how="left")

In [147]:
population_df = merged_population[['city_id','population','year']]

In [148]:
population_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [149]:
population_from_sql = pd.read_sql("population", con=connection_string) 

## Get_weather

In [151]:
cities = cities_from_sql["city"].to_list()

In [152]:
city_weather_df = gwd.get_weather_data(cities)


In [153]:
merged_weather_df = city_weather_df.merge(cities_from_sql,
                                   on = "city",
                                   how="left")

In [154]:
weather_df = merged_weather_df.drop(columns=["city","country_id", "latitude","longitude"])

In [155]:
weather_df['forecast_time']=pd.to_datetime(weather_df['forecast_time'])

In [156]:
weather_df.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)

120

In [157]:
weather_from_sql = pd.read_sql("weather", con=connection_string)

## Get_airports

In [159]:
city_latitude = cities_from_sql['latitude'].to_list()
city_longitude = cities_from_sql['longitude'].to_list()

In [160]:
airports_df = gac.get_airport_codes(city_latitude, city_longitude)

In [161]:
merged_airport_df = pd.merge(airports_df, cities_from_sql,
                                   on = ['latitude', 'longitude'],
                                   how="left")

In [162]:
airport_df= merged_airport_df.drop(columns=["latitude", "longitude","city","country_id"])

In [163]:
airport_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

8

In [164]:
airports_from_sql = pd.read_sql("airports", con=connection_string)

In [165]:
airport_icao = airports_from_sql['icao_code'].to_list()

## Get_flights

In [167]:
flights_df = gfa.get_flight_arrivals(airport_icao)

In [168]:
merged_flights_df = flights_df.merge(airports_from_sql,
                                   on = "icao_code",
                                   how="left")

In [169]:
flights_df = merged_flights_df.drop(columns=["icao_code","iata_code", "airport_name","city_name", "city_id"])

In [170]:
flights_df['arrival_times']=pd.to_datetime(flights_df['arrival_times'])

In [171]:
flights_df.to_sql('flights',
                   if_exists='append',
                   con=connection_string,
                   index=False)

417

In [172]:
flights_from_sql = pd.read_sql("flights", con=connection_string)

In [173]:
flights_from_sql

,flight_id,airlines,flight_numbers,arrival_times,arrival_terminals,airport_id
0,1,Aero Services Executive,W4 5105,2025-01-22 11:30:00,2,2
1,2,Sun Express,XQ 160,2025-01-22 11:35:00,1,2
2,3,Air Cairo,SM 2960,2025-01-22 11:35:00,1,2
3,4,Wizz Air,W6 2315,2025-01-22 11:35:00,2,2
4,5,Georgian,A9 781,2025-01-22 11:40:00,1,2
...,...,...,...,...,...,...
412,413,Lufthansa City,VL 1955,2025-01-22 21:10:00,2,8
413,414,Air France,AF 1122,2025-01-22 21:20:00,1,8
414,415,LOT - Polish,LO 355,2025-01-22 21:25:00,2,8
415,416,British,BA 958,2025-01-22 21:45:00,1,8
